In [1]:
import cairosvg
import os
from glob import glob
from PyPDF2 import PdfFileMerger
import time
from selenium import webdriver
import urllib.request
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.action_chains import ActionChains

# links
login_address='https://app.business-in-a-box.com/account/login'
# target_page='https://app.business-in-a-box.com/doc/business-plan-template-D12528'
# target_xml_doc='https://templates.biztree.com/svgs/docviewerWebApp.html?v4#12528.xml'
# one_sample_svg = 'https://templates.biztree.com/svgs/12528_files/page1.svg'

In [1]:
#Definning some basic functions for later usage
def clickOnId(id):
	browser.find_element_by_id(id).click()
	
def clickOnXpath(xpath):
	browser.find_element_by_xpath(xpath).click()

def clickOnClass(class_name):
	browser.find_element_by_class_name(class_name).click()

def TypeInId(id,toBeTyped):
	elems = browser.find_elements_by_id(id)
	elems[0].send_keys(toBeTyped)
	
def TypeInXpath(xpath,toBeTyped):
	elems = browser.find_elements_by_xpath(xpath)
	elems[0].send_keys(toBeTyped)
    
def openBrowser():
    chromeOptions = webdriver.ChromeOptions()
    chromedriver = "chromedriver.exe"
    global browser
    browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=chromeOptions)
    #browser.fullscreen_window()	
    return browser

def login_into_system():
    browser.get(login_address)
    TypeInXpath('//*[@id="Email"]',"your_BIB_email")
    TypeInXpath('//*[@id="Password"]',"your_BIB_password")
    clickOnXpath('//*[@id="BibWebApp"]/div[2]/div[2]/div/div[2]/div/div/form/button')
    time.sleep(3)
    
def take_screenshot(doc_id,field_name):
    # downloading the files into appropriate folder
    for idx in range(1,50):
        try:
            browser.get('https://templates.biztree.com/svgs/{0}_files/page{1}.svg'.format(doc_id,idx))
            urllib.request.urlretrieve('https://templates.biztree.com/svgs/{0}_files/page{1}.svg'.format(doc_id,idx),'{0}.svg'.format(idx))
        except Exception as ex:
            break
            
def pdf_converter(doc_id,field_name,working_directory):
    
    list_of_SVGs = glob("*.svg")
#     print(list_of_SVGs)
    for svg in list_of_SVGs:
        print(svg)
        try:
            cairosvg.svg2pdf(file_obj=open(str(svg), "rb"), write_to="{0}.pdf".format(svg.split(".")[0]))
            os.remove(svg)
        except Exception as URLError:
            pass
    merger = PdfFileMerger()
    
    list_of_PDFs = glob("*.pdf")
    
    new_pdfs = []
    for bimbil in list_of_PDFs:
        if "_" not in bimbil:
            new_pdfs.append(bimbil)
        new_pdfs.sort(key= lambda x: int(x.rsplit('.')[0].split("\\")[-1]))

    for pdf in new_pdfs:
        merger.append(pdf)
#     os.chdir(working_directory)
    merger.write("{0}_result.pdf".format(doc_id))
    merger.close()
    for pdf in new_pdfs:
        os.remove(pdf)


In [2]:
import pandas as pd
import os
df = pd.read_excel("list.xlsx")
organized_list = list()

for cat in pd.unique(df['Category']):
    print(cat)
#     os.mkdir(cat)
#     os.remove(cat)
#     organized_list.append(df[df['Category']==cat])


In [5]:
def main():
    
    browser=openBrowser()
    login_into_system()
    for index,category in enumerate(organized_list):
        specific_cat= category.iloc(0)[0]['Category']
        print("\n",index,specific_cat,"\n")
        for item in organized_list[index]['Document ID']:
            working_directory = os.getcwd() + '\\' + str(specific_cat)
            print(str(specific_cat), item,"in dir: {0}".format(working_directory))
            os.chdir(working_directory)
            take_screenshot(item,field_name=category)
            pdf_converter(item,field_name=category,working_directory=category)
            os.chdir('C:/Users/hades/Desktop/bussines_in_a_box')

        print()


In [ ]:
main()